This notebook allows collecting results across multiple pipeline runs. Concatenating across sessions and bin sizes. 
It takes CSVs, then determines the most recent one from the filename. 

There are two major sets of decoded epochs - Laps and Ripples/Replays
There are two sets of marginals for the decoded epochs - the "by epoch" and "by time bin" marginals.
The "by time bin" epochs are a larger granulation, with each epoch consisting of one or more time bin.


```python

# 2024-01-23 - 
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-fet11-01_12-58-54_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-11-03_12-3-25_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-11-02_17-46-44_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-11-02_19-28-0_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-4-10_12-58-3_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-4-10_12-25-50_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-4-09_16-40-54_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-09_22-24-40_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-12_16-53-46_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-4-09_17-29-30_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-08_14-26-15_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-09_1-22-43_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-07_16-40-19_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-08_21-16-25_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-12_15-55-31_time_bin_size_sweep_results.h5



# found_session_export_paths = [Path(v).resolve() for v in  ["C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0420PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0420PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0645PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0645PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0828PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0828PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv",
# ]]

[
    "2024-02-19_0430PM-kdiba_vvp01_one_2006-4-10_12-25-50-(ripple_simple_pf_pearson_merged_df)_tbin-0.025.csv",
    "2024-02-17_0210AM-kdiba_vvp01_one_2006-4-09_17-29-30-(laps_simple_pf_pearson_merged_df)_tbin-0.025.csv",
    "2024-02-19_0430PM-kdiba_vvp01_one_2006-4-10_12-25-50-(laps_weighted_corr_merged_df)_tbin-0.025.csv",
    "2024-02-19_0430PM-kdiba_vvp01_one_2006-4-10_12-25-50-(ripple_weighted_corr_merged_df)_tbin-0.025.csv",
]

```

In [11]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3

# required to enable non-blocking interaction:
%gui qt5

import sys
from copy import deepcopy
from typing import Dict, List, Tuple, Optional, Callable, Union, Any
from typing_extensions import TypeAlias
from nptyping import NDArray
import neuropy.utils.type_aliases as types
from collections import defaultdict

import numpy as np
import pandas as pd
import re
from pathlib import Path
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Plotting
# import pylustrator # customization of figures
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()

matplotlib.use('Qt5Agg')
# %matplotlib inline
# %matplotlib auto

# Switch to the desired interactivity mode
plt.interactive(True)

# _restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# _restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
import seaborn as sns

# from ..PendingNotebookCode import plot_across_sessions_scatter_results, plot_histograms, plot_stacked_histograms
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import find_csv_files, find_HDF5_files, find_most_recent_files, process_csv_file, plot_across_sessions_scatter_results, plot_histograms, plot_stacked_histograms
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DecoderDecodedEpochsResult

debug_print: bool = False

TODAY_DAY_DATE: str = f"2024-03-02"

print(f'TODAY_DAY_DATE: {TODAY_DAY_DATE}')

types.session_str: TypeAlias = str # a unique session identifier



Automatic pdb calling has been turned OFF
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
TODAY_DAY_DATE: 2024-03-02


# Definitions
"_new" (as in `all_sessions_new_ripple_df`) adds simple_pearson and a few other measures

In [15]:
## Load across session t_delta CSV, which contains the t_delta for each session:

# t_delta_csv_path = Path(r'C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\output\collected_outputs\2024-01-18_GL_t_split_df.csv').resolve() # Apogee
# t_delta_csv_path = Path('/home/halechr/cloud/turbo/Data/Output/collected_outputs/2024-01-18_GL_t_split_df.csv').resolve() # GL

# collected_outputs_directory = '/home/halechr/FastData/collected_outputs/'
# collected_outputs_directory = r'C:\Users\pho\Desktop\collected_outputs'
# collected_outputs_directory = r'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs' # APOGEE
# collected_outputs_directory = '/home/halechr/cloud/turbo/Data/Output/collected_outputs' # GL

known_collected_outputs_paths = [Path(v).resolve() for v in [r'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs', '/home/halechr/cloud/turbo/Data/Output/collected_outputs', '/home/halechr/FastData/collected_outputs/']]
collected_outputs_directory = find_first_extant_path(known_collected_outputs_paths)
assert collected_outputs_directory.exists(), f"collected_outputs_directory: {collected_outputs_directory} does not exist! Is the right computer's config commented out above?"
# fullwidth_path_widget(scripts_output_path, file_name_label='Scripts Output Path:')
print(f'collected_outputs_directory: {collected_outputs_directory}')

t_delta_csv_path = collected_outputs_directory.joinpath('2024-01-18_GL_t_split_df.csv').resolve() # GL
assert t_delta_csv_path.exists()

## Find the files:
csv_files = find_csv_files(collected_outputs_directory)
h5_files = find_HDF5_files(collected_outputs_directory)

csv_sessions = find_most_recent_files(found_session_export_paths=csv_files)
h5_sessions = find_most_recent_files(found_session_export_paths=h5_files)

## The CSV containing the session delta time:
t_delta_df = pd.read_csv(t_delta_csv_path, index_col=0) # Assuming that your CSV file has an index column
# adds `delta_aligned_t_start`, `delta_aligned_t_end` columns
t_delta_df['delta_aligned_t_start'] = t_delta_df['t_start'] - t_delta_df['t_delta']
t_delta_df['delta_aligned_t_end'] = t_delta_df['t_end'] - t_delta_df['t_delta']

# computes `earliest_delta_aligned_t_start`, latest_delta_aligned_t_end
earliest_delta_aligned_t_start: float = np.nanmin(t_delta_df['delta_aligned_t_start'])
latest_delta_aligned_t_end: float = np.nanmax(t_delta_df['delta_aligned_t_end'])
print(f'earliest_delta_aligned_t_start: {earliest_delta_aligned_t_start}, latest_delta_aligned_t_end: {latest_delta_aligned_t_end}')
t_delta_dict = t_delta_df.to_dict(orient='index')
# t_delta_df


# #TODO 2024-03-02 12:12: - [ ] Could add weighted correlation if there is a dataframe for that and it's computed:
_df_raw_variable_names = ['simple_pf_pearson_merged_df', 'weighted_corr_merged_df']
_df_variables_names = ['laps_weighted_corr_merged_df', 'ripple_weighted_corr_merged_df', 'laps_simple_pf_pearson_merged_df', 'ripple_simple_pf_pearson_merged_df']

# # tbin_values_dict = {'laps': self.laps_decoding_time_bin_size, 'ripple': self.ripple_decoding_time_bin_size}
time_col_name_dict = {'laps': 'lap_start_t', 'ripple': 'ripple_start_t'} ## default should be 't_bin_center'

# fold older files:
# {'laps_marginals_df': 'lap_start_t', 'ripple_marginals_df': 'ripple_start_t', 'laps_time_bin_marginals_df':'t_bin_center', 'ripple_time_bin_marginals_df':'t_bin_center'}
    

def process_and_load_file(session_dict, df_file_name_key, loaded_dict, session_name, curr_session_t_delta, time_key):
    try:
        file_path = session_dict[df_file_name_key]
        loaded_dict[session_name] = process_csv_file(file_path, session_name, curr_session_t_delta, time_key)
    except BaseException as e:
        print(f'session {session_name} did not fully work. (error {e}. Skipping.')


# def prototype_flexible_process_and_load_file(session_dict, df_file_name_key, loaded_dict, session_name, curr_session_t_delta, time_key):
#     """ supposed to not require a bunch of manually created input variable names like:

#     final_sessions_loaded_laps_dict = {}
#     final_sessions_loaded_ripple_dict = {}
#     final_sessions_loaded_laps_time_bin_dict = {}
#     final_sessions_loaded_ripple_time_bin_dict = {}

#     and then processing them separately

#     Instead it splits the df_file_name_key like 'ripple_weighted_corr_merged_df' -> ('ripple', 'weighted_corr_merged_df')
#         and assigns to a nested dictionary
    
#     """
#     (an_epochs_source_name, a_df_variable_name) = df_file_name_key.split(sep='_', maxsplit=1) # get the first part of the variable names that indicates whether it's for "laps" or "ripple"
#     # like 'ripple_weighted_corr_merged_df' -> ('ripple', 'weighted_corr_merged_df')
#     try:
#         file_path = session_dict[df_file_name_key]
#         if an_epochs_source_name not in loaded_dict[session_name]:
#             loaded_dict[session_name][an_epochs_source_name] = {}
#         loaded_dict[session_name][an_epochs_source_name][a_df_variable_name] = process_csv_file(file_path, session_name, curr_session_t_delta, time_key)
#     except BaseException as e:
#         print(f'session {session_name} did not fully work. (error {e}. Skipping.')


# csv_sessions
# Extract each of the separate files from the sessions:

# final_sessions: Dict[types.session_str, Dict[str, Path]] = {}

final_sessions_loaded_laps_dict = {}
final_sessions_loaded_ripple_dict = {}
final_sessions_loaded_laps_time_bin_dict = {}
final_sessions_loaded_ripple_time_bin_dict = {}

final_sessions_loaded_laps_new_dict = {}
final_sessions_loaded_ripple_new_dict = {}

# stupid reudndant but compatible method
final_sessions_loaded_laps_wcorr_dict = {}
final_sessions_loaded_ripple_wcorr_dict = {}


# final_sessions_loaded_nested_dict = {} # to be filled like
# {
#     'laps': {},
#     'ripple': {},
# }

final_sessions: Dict[types.session_str, Dict[str, Path]] = {session_str:{file_type:a_path for file_type, (an_decoding_time_bin_size_str, a_path, an_export_datetime) in session_dict.items()}
                                                                                          for session_str, session_dict in csv_sessions.items()}

for session_str, session_dict in final_sessions.items():
    session_name = str(session_str)  # Extract session name from the filename
    if debug_print:
        print(f'processing session_name: {session_name}')
    curr_session_t_delta = t_delta_dict.get(session_name, {}).get('t_delta', None)
    if curr_session_t_delta is None:
        print(f'WARN: curr_session_t_delta is None for session_str = "{session_str}"')

    # Process each file type with its corresponding details
    process_and_load_file(session_dict, 'laps_marginals_df', final_sessions_loaded_laps_dict, session_str, curr_session_t_delta, 'lap_start_t')
    process_and_load_file(session_dict, 'ripple_marginals_df', final_sessions_loaded_ripple_dict, session_str, curr_session_t_delta, 'ripple_start_t')
    process_and_load_file(session_dict, 'laps_time_bin_marginals_df', final_sessions_loaded_laps_time_bin_dict, session_str, curr_session_t_delta, 't_bin_center')
    process_and_load_file(session_dict, 'ripple_time_bin_marginals_df', final_sessions_loaded_ripple_time_bin_dict, session_str, curr_session_t_delta, 't_bin_center')
    process_and_load_file(session_dict, 'laps_simple_pf_pearson_merged_df', final_sessions_loaded_laps_new_dict, session_str, curr_session_t_delta, 'lap_start_t')
    process_and_load_file(session_dict, 'ripple_simple_pf_pearson_merged_df', final_sessions_loaded_ripple_new_dict, session_str, curr_session_t_delta, 'ripple_start_t')
    process_and_load_file(session_dict, 'laps_weighted_corr_merged_df', final_sessions_loaded_laps_wcorr_dict, session_str, curr_session_t_delta, 'lap_start_t')
    process_and_load_file(session_dict, 'ripple_weighted_corr_merged_df', final_sessions_loaded_ripple_wcorr_dict, session_str, curr_session_t_delta, 'ripple_start_t')

    # prototype_flexible_process_and_load_file(session_dict, 'laps_marginals_df', final_sessions_loaded_nested_dict, session_str, curr_session_t_delta, 'lap_start_t')
    # prototype_flexible_process_and_load_file(session_dict, 'ripple_marginals_df', final_sessions_loaded_nested_dict, session_str, curr_session_t_delta, 'ripple_start_t')
    # prototype_flexible_process_and_load_file(session_dict, 'laps_time_bin_marginals_df', final_sessions_loaded_nested_dict, session_str, curr_session_t_delta, 't_bin_center')
    # prototype_flexible_process_and_load_file(session_dict, 'ripple_time_bin_marginals_df', final_sessions_loaded_nested_dict, session_str, curr_session_t_delta, 't_bin_center')
        
    # prototype_flexible_process_and_load_file(session_dict, 'laps_simple_pf_pearson_merged_df', final_sessions_loaded_nested_dict, session_str, curr_session_t_delta, 'lap_start_t')
    # prototype_flexible_process_and_load_file(session_dict, 'ripple_simple_pf_pearson_merged_df', final_sessions_loaded_nested_dict, session_str, curr_session_t_delta, 'ripple_start_t')

    # prototype_flexible_process_and_load_file(session_dict, 'laps_weighted_corr_merged_df', final_sessions_loaded_nested_dict, session_str, curr_session_t_delta, 'lap_start_t')
    # prototype_flexible_process_and_load_file(session_dict, 'ripple_weighted_corr_merged_df', final_sessions_loaded_nested_dict, session_str, curr_session_t_delta, 'ripple_start_t')


# for session_str, session_dict in csv_sessions.items():
#     final_sessions[session_str] = {}
#     # final_sessions_loaded_nested_dict[session_str] = {
#     #     'laps': {},
#     #     'ripple': {},
#     # }
    
#     for file_type, (an_decoding_time_bin_size_str, a_path, an_export_datetime) in session_dict.items():
#         final_sessions[session_str][file_type] = a_path
        
#     session_name: str = str(session_str)  # Extract session name from the filename
#     if debug_print:
#         print(f'processing session_name: {session_name}')
#     curr_session_t_delta: Optional[float] = t_delta_dict.get(session_name, {}).get('t_delta', None)
#     if curr_session_t_delta is None:
#         print(f'WARN: curr_session_t_split is None for session_str = "{session_str}"')

#     # finds each of the four exports:
#     try:
#         laps_file = final_sessions[session_str]['laps_marginals_df']
#         final_sessions_loaded_laps_dict[session_str] = process_csv_file(laps_file, session_name, curr_session_t_delta, 'lap_start_t')
#     except BaseException as e:
#         print(f'session {session_str} did not fully work. (error {e}. Skipping.') 
#         pass
                
#     try:
#         ripple_file = final_sessions[session_str]['ripple_marginals_df']
#         final_sessions_loaded_ripple_dict[session_str] = process_csv_file(ripple_file, session_name, curr_session_t_delta, 'ripple_start_t')
#     except BaseException as e:
#         print(f'session {session_str} did not fully work. (error {e}. Skipping.') 
#         pass
        
#     try:
#         laps_time_bin_file = final_sessions[session_str]['laps_time_bin_marginals_df']
#         final_sessions_loaded_laps_time_bin_dict[session_str] = process_csv_file(laps_time_bin_file, session_name, curr_session_t_delta, 't_bin_center')
#     except BaseException as e:
#         print(f'session {session_str} did not fully work. (error {e}. Skipping.') 
#         pass   

#     try:
#         ripple_time_bin_file = final_sessions[session_str]['ripple_time_bin_marginals_df']
#         final_sessions_loaded_ripple_time_bin_dict[session_str] = process_csv_file(ripple_time_bin_file, session_name, curr_session_t_delta, 't_bin_center')
#     except BaseException as e:
#         print(f'session {session_str} did not fully work. (error {e}. Skipping.') 
#         pass

#     # 2024-02-19 NEW laps_simple_pf_pearson_merged_df ____________________________________________________________________ #
#     # Allowed time_bin_size = 0.025
#     try:
#         laps_file = final_sessions[session_str]['laps_simple_pf_pearson_merged_df']
#         final_sessions_loaded_laps_new_dict[session_str] = process_csv_file(laps_file, session_name, curr_session_t_delta, 'lap_start_t')
#     except BaseException as e:
#         print(f'session {session_str} did not fully work. (error {e}. Skipping.') 
#         pass
                
#     try:
#         ripple_file = final_sessions[session_str]['ripple_simple_pf_pearson_merged_df']
#         final_sessions_loaded_ripple_new_dict[session_str] = process_csv_file(ripple_file, session_name, curr_session_t_delta, 'ripple_start_t')
#     except BaseException as e:
#         print(f'session {session_str} did not fully work. (error {e}. Skipping.') 
#         pass
    
#     try:
#         ripple_file = final_sessions[session_str]['ripple_weighted_corr_merged_df']
#         final_sessions_loaded_ripple_new_dict[session_str] = process_csv_file(ripple_file, session_name, curr_session_t_delta, 'ripple_start_t')
#     except BaseException as e:
#         print(f'session {session_str} did not fully work. (error {e}. Skipping.') 
#         pass
    
        
    # ripple_weighted_corr_merged_df
    # ['ripple_weighted_corr_merged_df']

## Build across_sessions join dataframes:
all_sessions_laps_df: pd.DataFrame = pd.concat(list(final_sessions_loaded_laps_dict.values()), axis='index', ignore_index=True)
all_sessions_ripple_df: pd.DataFrame = pd.concat(list(final_sessions_loaded_ripple_dict.values()), axis='index', ignore_index=True)
# Add 'epoch_idx' column for compatibility:
all_sessions_laps_df['epoch_idx'] = all_sessions_laps_df['lap_idx']
all_sessions_ripple_df['epoch_idx'] = all_sessions_ripple_df['ripple_idx']

# *_time_bin marginals:
all_sessions_laps_time_bin_df: pd.DataFrame = pd.concat(list(final_sessions_loaded_laps_time_bin_dict.values()), axis='index', ignore_index=True)
all_sessions_ripple_time_bin_df: pd.DataFrame = pd.concat(list(final_sessions_loaded_ripple_time_bin_dict.values()), axis='index', ignore_index=True)

# NEW ________________________________________________________________________________________________________________ #
all_sessions_new_laps_df: pd.DataFrame = pd.concat(list(final_sessions_loaded_laps_new_dict.values()), axis='index', ignore_index=True)
all_sessions_new_ripple_df: pd.DataFrame = pd.concat(list(final_sessions_loaded_ripple_new_dict.values()), axis='index', ignore_index=True)

if len(final_sessions_loaded_laps_wcorr_dict) > 0:
    all_sessions_wcorr_laps_df: pd.DataFrame = pd.concat(list(final_sessions_loaded_laps_wcorr_dict.values()), axis='index', ignore_index=True)
else:
    all_sessions_wcorr_laps_df = None # empty df would be better

all_sessions_wcorr_ripple_df: pd.DataFrame = pd.concat(list(final_sessions_loaded_ripple_wcorr_dict.values()), axis='index', ignore_index=True)


if 'time_bin_size' not in all_sessions_laps_df:
    print('Uh-oh! time_bin_size is missing! This must be old exports!')
    print(f'\tTry to determine the time_bin_size from the filenames: {csv_sessions}')
    ## manual correction UwU
    time_bin_size: float = 0.025
    print(f'WARNING! MANUAL OVERRIDE TIME BIN SIZE SET: time_bin_size = {time_bin_size}. Assigning to dataframes....')
    all_sessions_laps_df['time_bin_size'] = time_bin_size
    all_sessions_ripple_df['time_bin_size'] = time_bin_size
    all_sessions_laps_time_bin_df['time_bin_size'] = time_bin_size
    all_sessions_ripple_time_bin_df['time_bin_size'] = time_bin_size
    print(f'\tdone.')
else:
    # Filter rows based on column: 'time_bin_size'
    all_sessions_laps_df = all_sessions_laps_df[all_sessions_laps_df['time_bin_size'].notna()]
    all_sessions_ripple_df = all_sessions_ripple_df[all_sessions_ripple_df['time_bin_size'].notna()]
    all_sessions_laps_time_bin_df = all_sessions_laps_time_bin_df[all_sessions_laps_time_bin_df['time_bin_size'].notna()]
    all_sessions_ripple_time_bin_df = all_sessions_ripple_time_bin_df[all_sessions_ripple_time_bin_df['time_bin_size'].notna()]

def _common_cleanup_operations(a_df):
    if a_df is None:
        return None
    ## Drop the weird 'Unnamed: 0' column:
    # Rename column 'Unnamed: 0' to 'abs_time_bin_index'
    a_df = a_df.rename(columns={'Unnamed: 0': 'abs_time_bin_index'})
    # Drop column: 'abs_time_bin_index'
    a_df = a_df.drop(columns=['abs_time_bin_index'])
    # Add additional 'epoch_idx' column for compatibility:
    if 'epoch_idx' not in a_df:
        if 'lap_idx' in a_df:
            a_df['epoch_idx'] = a_df['lap_idx']
        if 'ripple_idx' in a_df:
            a_df['epoch_idx'] = a_df['ripple_idx']
    return a_df


all_sessions_laps_df, all_sessions_ripple_df, all_sessions_laps_time_bin_df, all_sessions_ripple_time_bin_df = [_common_cleanup_operations(a_df) for a_df in (all_sessions_laps_df, all_sessions_ripple_df, all_sessions_laps_time_bin_df, all_sessions_ripple_time_bin_df)]
all_sessions_new_laps_df, all_sessions_new_ripple_df, all_sessions_wcorr_laps_df, all_sessions_wcorr_ripple_df = [_common_cleanup_operations(a_df) for a_df in (all_sessions_new_laps_df, all_sessions_new_ripple_df, all_sessions_wcorr_laps_df, all_sessions_wcorr_ripple_df)]

all_sessions_new_laps_df: pd.DataFrame = DecoderDecodedEpochsResult.merge_decoded_epochs_result_dfs(all_sessions_new_laps_df, all_sessions_wcorr_laps_df, should_drop_directional_columns=False, start_t_idx_name='delta_aligned_start_t')
all_sessions_new_ripple_df: pd.DataFrame = DecoderDecodedEpochsResult.merge_decoded_epochs_result_dfs(all_sessions_new_ripple_df, all_sessions_wcorr_ripple_df, should_drop_directional_columns=False, start_t_idx_name='ripple_start_t')

# all_sessions_laps_time_bin_df # 601845 rows × 9 column

collected_outputs_directory: C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\output\collected_outputs
ERR: Could not parse filename: "2024-01-18_GL_t_split_df"
earliest_delta_aligned_t_start: -2057.2259484970764, latest_delta_aligned_t_end: 1661.8560019930592
session kdiba_vvp01_two_2006-4-10_12-58-3 did not fully work. (error 'laps_simple_pf_pearson_merged_df'. Skipping.
session kdiba_vvp01_two_2006-4-10_12-58-3 did not fully work. (error 'ripple_simple_pf_pearson_merged_df'. Skipping.
session kdiba_vvp01_two_2006-4-10_12-58-3 did not fully work. (error 'laps_weighted_corr_merged_df'. Skipping.
session kdiba_vvp01_two_2006-4-10_12-58-3 did not fully work. (error 'ripple_weighted_corr_merged_df'. Skipping.
session kdiba_pin01_one_fet11-01_12-58-54 did not fully work. (error 'laps_simple_pf_pearson_merged_df'. Skipping.
session kdiba_pin01_one_fet11-01_12-58-54 did not fully work. (error 'ripple_simple_pf_pearson_merged_df'. Skipping.
session kdiba_pin01_one_fet11-01_12-58-54 did not fully wo

In [16]:
all_sessions_new_ripple_df # 3138 rows × 24 columns

,P_LR,P_RL,P_Long,P_Short,ripple_idx,ripple_start_t,P_Long_LR,P_Long_RL,P_Short_LR,P_Short_RL,...,long_best_pf_peak_x_pearsonr,short_best_pf_peak_x_pearsonr,wcorr_long_LR,wcorr_long_RL,wcorr_short_LR,wcorr_short_RL,long_best_wcorr,short_best_wcorr,wcorr_abs_diff,pearsonr_abs_diff
0,0.629641,0.370359,0.701551,0.298449,0,40.187177,0.441725,0.259826,0.187916,0.110533,...,NaN,NaN,0.247980,0.146485,0.132055,0.238241,0.146485,0.238241,-0.091756,NaN
1,0.470906,0.529094,0.518903,0.481097,1,41.011916,0.244354,0.274548,0.226551,0.254546,...,-0.195129,-0.218456,0.018191,-0.187365,-0.056323,-0.073510,-0.187365,-0.073510,0.113855,-0.023327
2,0.499931,0.500069,0.877526,0.122474,2,43.429702,0.438702,0.438824,0.061229,0.061246,...,NaN,NaN,-0.664704,-0.254211,-0.762843,-0.331398,-0.254211,-0.331398,-0.077187,NaN
3,0.788521,0.211479,0.671782,0.328218,3,44.189868,0.529714,0.142068,0.258806,0.069411,...,NaN,NaN,0.215749,-0.791509,-0.633169,-0.324092,-0.791509,-0.324092,0.467418,NaN
4,0.401167,0.598833,0.591382,0.408618,4,44.588736,0.237243,0.354139,0.163924,0.244694,...,-0.529164,-0.462075,-0.288470,-0.434733,-0.139969,-0.614672,-0.434733,-0.614672,-0.179939,0.067089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3133,0.530453,0.469547,0.328807,0.671193,116,1091.508966,0.174417,0.154390,0.356037,0.315157,...,-0.146854,-0.144738,0.704046,-0.425856,-0.043927,-0.394177,0.704046,-0.043927,0.660118,0.002116
3134,0.497632,0.502368,0.400255,0.599745,117,1152.140016,0.199179,0.201075,0.298452,0.301293,...,0.369946,0.222189,-0.100364,-0.076447,-0.138187,-0.153548,-0.100364,-0.138187,-0.037823,0.147757
3135,0.412875,0.587125,0.454412,0.545588,118,1152.976306,0.187615,0.266797,0.225260,0.320329,...,NaN,NaN,0.188139,0.024410,0.114085,-0.184394,0.188139,0.114085,0.074053,NaN
3136,0.470149,0.529851,0.588251,0.411749,119,1167.479548,0.276566,0.311686,0.193583,0.218165,...,NaN,NaN,0.109532,0.247803,-0.113319,0.014584,0.247803,0.014584,0.233220,NaN


In [17]:
all_sessions_new_laps_df # 931 rows × 25 columns

,P_LR,P_RL,P_Long,P_Short,lap_idx,lap_start_t,P_Long_LR,P_Long_RL,P_Short_LR,P_Short_RL,...,long_best_pf_peak_x_pearsonr,short_best_pf_peak_x_pearsonr,wcorr_long_LR,wcorr_long_RL,wcorr_short_LR,wcorr_short_RL,long_best_wcorr,short_best_wcorr,wcorr_abs_diff,pearsonr_abs_diff
0,0.700788,0.299212,0.601958,0.398042,0,5.635867,0.421845,0.180113,0.278943,0.119099,...,-0.502526,-0.524721,-0.411687,-0.216455,-0.292481,-0.212025,-0.216455,-0.212025,0.004430,-0.022195
1,0.296269,0.703731,0.733725,0.266275,1,31.862536,0.217380,0.516345,0.078889,0.187386,...,0.699403,0.668086,0.449495,0.569257,0.345957,0.435315,0.569257,0.435315,0.133943,0.031317
2,0.702421,0.297579,0.625274,0.374726,2,135.801698,0.439205,0.186069,0.263215,0.111511,...,-0.514232,-0.468185,-0.478622,-0.217475,-0.328050,-0.186347,-0.217475,-0.186347,0.031127,0.046047
3,0.256863,0.743137,0.756906,0.243094,3,161.458825,0.194421,0.562485,0.062442,0.180652,...,0.773049,0.751156,0.475672,0.626081,0.405159,0.489848,0.626081,0.489848,0.136233,0.021892
4,0.698606,0.301394,0.563065,0.436935,4,234.465983,0.393361,0.169704,0.305246,0.131689,...,-0.344997,-0.394734,-0.284786,-0.062499,-0.138034,-0.094640,-0.062499,-0.094640,-0.032141,-0.049738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926,0.345465,0.654535,0.420308,0.579692,89,1137.400928,0.145202,0.275107,0.200263,0.379429,...,-0.299085,-0.527674,-0.109075,-0.399972,-0.109503,-0.255732,-0.109075,-0.109503,-0.000429,-0.228588
927,0.603277,0.396723,0.413195,0.586805,90,1142.779242,0.249271,0.163924,0.354006,0.232799,...,0.736108,0.756790,0.322863,0.089369,0.261810,0.087887,0.322863,0.261810,0.061053,-0.020682
928,0.338918,0.661082,0.423155,0.576845,91,1147.879397,0.143415,0.279740,0.195503,0.381342,...,-0.220436,-0.451283,-0.066119,-0.407022,-0.103891,-0.276141,-0.066119,-0.103891,-0.037772,-0.230847
929,0.601921,0.398079,0.394573,0.605427,92,1153.984058,0.237502,0.157071,0.364419,0.241008,...,0.852999,0.870789,0.394794,0.072800,0.315574,0.142393,0.394794,0.315574,0.079220,-0.017790


In [18]:
all_sessions_ripple_df # 17592 rows × 10 columns

,P_LR,P_RL,P_Long,P_Short,ripple_idx,ripple_start_t,session_name,delta_aligned_start_t,time_bin_size,epoch_idx
0,4.291495e-01,0.570850,7.378908e-01,0.262109,0,40.187177,kdiba_gor01_one_2006-6-08_14-26-15,-1171.370903,0.03,0
1,4.651143e-01,0.534886,5.144524e-01,0.485548,1,41.011916,kdiba_gor01_one_2006-6-08_14-26-15,-1170.546164,0.03,1
2,7.273230e-01,0.272677,4.339393e-01,0.566061,2,44.588736,kdiba_gor01_one_2006-6-08_14-26-15,-1166.969344,0.03,2
3,1.566807e-02,0.984332,8.318798e-01,0.168120,3,45.379714,kdiba_gor01_one_2006-6-08_14-26-15,-1166.178366,0.03,3
4,5.496904e-01,0.450310,7.779977e-01,0.222002,4,45.994329,kdiba_gor01_one_2006-6-08_14-26-15,-1165.563751,0.03,4
...,...,...,...,...,...,...,...,...,...,...
18305,9.510568e-01,0.048943,3.457463e-02,0.965425,72,747.888923,kdiba_gor01_two_2006-6-12_16-53-46,276.821522,0.10,72
18306,6.233424e-01,0.376658,5.285045e-01,0.471496,73,753.268361,kdiba_gor01_two_2006-6-12_16-53-46,282.200960,0.10,73
18307,6.717797e-01,0.328220,9.305013e-01,0.069499,74,764.421804,kdiba_gor01_two_2006-6-12_16-53-46,293.354403,0.10,74
18308,1.530999e-09,1.000000,1.530994e-09,1.000000,75,779.004365,kdiba_gor01_two_2006-6-12_16-53-46,307.936965,0.10,75


In [ ]:
csv_files

In [19]:
final_output_path = Path("../output/").resolve()

# final_sessions
# {'kdiba_gor01_one_2006-6-08_14-26-15': {'ripple_marginals_df': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-17_0540PM-kdiba_gor01_one_2006-6-08_14-26-15-(ripple_marginals_df).csv'),
#   'laps_marginals_df': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-17_0540PM-kdiba_gor01_one_2006-6-08_14-26-15-(laps_marginals_df).csv')},
#  'kdiba_gor01_one_2006-6-09_1-22-43': {'ripple_marginals_df': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0838PM-kdiba_gor01_one_2006-6-09_1-22-43-(ripple_marginals_df).csv'),
#   'laps_marginals_df': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0838PM-kdiba_gor01_one_2006-6-09_1-22-43-(laps_marginals_df).csv')},
#  'kdiba_pin01_one_fet11-01_12-58-54': {'ripple_marginals_df': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0828PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv'),
#   'laps_marginals_df': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0828PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv')}}

# Save out the four dataframes to CSVs:
final_dfs_dict = {"AcrossSession_Laps_per-Epoch": all_sessions_laps_df, "AcrossSession_Ripple_per-Epoch": all_sessions_ripple_df,
				   "AcrossSession_Laps_per-TimeBin": all_sessions_laps_time_bin_df, "AcrossSession_Ripple_per-TimeBin": all_sessions_ripple_time_bin_df,
				   "AcrossSession_NEW_Laps_per-Epoch": all_sessions_new_laps_df, "AcrossSession_NEW_Ripple_per-Epoch": all_sessions_new_ripple_df,
				   }

final_csv_export_paths = {}
for a_name, a_final_df in final_dfs_dict.items():
	# save out one final DF to csv.
	out_csv_filename: str = f"{TODAY_DAY_DATE}_{a_name}.csv"
	a_final_csv_export_path = final_output_path.joinpath(out_csv_filename).resolve()
	a_final_df.to_csv(a_final_csv_export_path) # save to CSV.
	final_csv_export_paths[a_name] = a_final_csv_export_path
	
final_csv_export_paths


{'AcrossSession_Laps_per-Epoch': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-03-02_AcrossSession_Laps_per-Epoch.csv'),
 'AcrossSession_Ripple_per-Epoch': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-03-02_AcrossSession_Ripple_per-Epoch.csv'),
 'AcrossSession_Laps_per-TimeBin': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-03-02_AcrossSession_Laps_per-TimeBin.csv'),
 'AcrossSession_Ripple_per-TimeBin': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-03-02_AcrossSession_Ripple_per-TimeBin.csv'),
 'AcrossSession_NEW_Laps_per-Epoch': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-03-02_AcrossSession_NEW_Laps_per-Epoch.csv'),
 'AcrossSession_NEW_Ripple_per-Epoch': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-03-02_AcrossSession_NEW_Ripple_per-Epoch.csv')}

In [ ]:
all_sessions_new_laps_df, all_sessions_new_ripple_df

In [ ]:
all_sessions_laps_df

# 2024-03-02 - Get only the user-annotated ripples:

In [26]:
# all_sessions_new_ripple_df['is_user_annotated_epoch'].notna()
# all_sessions_user_annotated_laps_df = all_sessions_new_laps_df.dropna(axis='index', subset=['is_user_annotated_epoch'], inplace=False) # user doesn't annotate laps
all_sessions_user_annotated_ripple_df = all_sessions_new_ripple_df.dropna(axis='index', subset=['is_user_annotated_epoch'], inplace=False)
all_sessions_user_annotated_ripple_df

,P_LR,P_RL,P_Long,P_Short,ripple_idx,ripple_start_t,P_Long_LR,P_Long_RL,P_Short_LR,P_Short_RL,...,long_best_pf_peak_x_pearsonr,short_best_pf_peak_x_pearsonr,wcorr_long_LR,wcorr_long_RL,wcorr_short_LR,wcorr_short_RL,long_best_wcorr,short_best_wcorr,wcorr_abs_diff,pearsonr_abs_diff
735,0.445777,0.554223,0.703199,0.296801,0,49.107308,0.313470,0.389729,0.132307,0.164494,...,NaN,NaN,-0.140748,0.077865,-0.079812,0.041526,0.077865,0.041526,0.036339,NaN
736,0.872211,0.127789,0.848804,0.151196,1,64.083055,0.740336,0.108467,0.131875,0.019321,...,NaN,NaN,-0.232588,-0.179068,-0.403697,-0.143706,-0.179068,-0.143706,0.035361,NaN
737,0.442045,0.557955,0.785592,0.214408,2,64.805681,0.347267,0.438325,0.094778,0.119630,...,NaN,NaN,-0.410401,-0.429249,-0.400531,0.048810,-0.429249,0.048810,0.380439,NaN
738,0.159710,0.840290,0.838429,0.161571,3,66.661598,0.133906,0.704523,0.025805,0.135767,...,-0.623696,-0.512350,0.643677,-0.747099,0.221974,-0.772675,-0.747099,-0.772675,-0.025577,0.111346
739,0.307556,0.692444,0.462155,0.537845,4,67.966460,0.142139,0.320017,0.165417,0.372427,...,-0.149586,-0.165016,0.578716,-0.678734,-0.225543,-0.709997,0.578716,-0.225543,0.353173,-0.015430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2887,0.762801,0.237199,0.281345,0.718655,702,2530.589578,0.214611,0.066735,0.548191,0.170464,...,-0.356116,-0.114637,0.439406,0.244648,0.207299,0.070563,0.439406,0.207299,0.232106,0.241479
2888,0.255017,0.744983,0.546816,0.453184,703,2531.245726,0.139447,0.407369,0.115570,0.337614,...,0.142335,0.101472,-0.462071,-0.256684,-0.485693,-0.270575,-0.256684,-0.270575,-0.013891,0.040862
2889,0.319081,0.680919,0.641305,0.358695,704,2533.065332,0.204628,0.436677,0.114453,0.244242,...,NaN,NaN,-0.412279,-0.042209,-0.146345,-0.162995,-0.042209,-0.162995,-0.120786,NaN
2890,0.999900,0.000100,0.610545,0.389455,705,2534.629471,0.610484,0.000061,0.389416,0.000039,...,NaN,NaN,-0.776638,-0.675438,-0.811339,-0.313954,-0.675438,-0.313954,0.361484,NaN


In [21]:
## want to compare difference between user approved and user rejected ripples

from pyphocorehelpers.indexing_helpers import partition_df

partitioned_dfs = dict(zip(*partition_df(all_sessions_user_annotated_ripple_df, partitionColumn='is_user_annotated_epoch')))

user_approved_ripple_df: pd.DataFrame = partitioned_dfs[True].drop(columns=['is_user_annotated_epoch']).reset_index(drop=True)
user_rejected_ripple_df: pd.DataFrame = partitioned_dfs[False].drop(columns=['is_user_annotated_epoch']).reset_index(drop=True)

user_approved_ripple_df


,P_LR,P_RL,P_Long,P_Short,ripple_idx,ripple_start_t,P_Long_LR,P_Long_RL,P_Short_LR,P_Short_RL,...,long_best_pf_peak_x_pearsonr,short_best_pf_peak_x_pearsonr,wcorr_long_LR,wcorr_long_RL,wcorr_short_LR,wcorr_short_RL,long_best_wcorr,short_best_wcorr,wcorr_abs_diff,pearsonr_abs_diff
0,0.159710,0.840290,0.838429,0.161571,3,66.661598,0.133906,0.704523,0.025805,0.135767,...,-0.623696,-0.512350,0.643677,-0.747099,0.221974,-0.772675,-0.747099,-0.772675,-0.025577,0.111346
1,0.634361,0.365639,0.783215,0.216785,42,292.623671,0.496841,0.286374,0.137520,0.079265,...,-0.478802,-0.471114,0.935025,0.932670,0.863378,0.919826,0.932670,0.919826,0.012845,0.007689
2,0.493034,0.506966,0.570170,0.429830,43,304.440267,0.281113,0.289057,0.211921,0.217909,...,-0.775220,-0.640006,0.139927,0.386869,0.110647,0.088710,0.386869,0.088710,0.298158,0.135214
3,0.188260,0.811740,0.790949,0.209051,51,380.745850,0.148904,0.642045,0.039356,0.169695,...,0.855873,0.689429,-0.849326,-0.882094,-0.682860,-0.853833,-0.882094,-0.853833,0.028261,0.166444
4,0.286592,0.713408,0.711961,0.288039,79,487.205157,0.204043,0.507919,0.082550,0.205489,...,-0.938163,-0.901337,-0.941448,-0.934071,-0.799006,-0.933250,-0.934071,-0.933250,0.000822,0.036825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,0.113030,0.886970,0.531551,0.468449,624,2261.095905,0.060081,0.471470,0.052949,0.415500,...,NaN,NaN,-0.672955,-0.736671,-0.301308,0.025316,-0.736671,0.025316,0.711355,NaN
230,0.116618,0.883382,0.615333,0.384667,626,2265.076969,0.071759,0.543574,0.044859,0.339808,...,NaN,NaN,-0.926538,-0.806767,-0.804224,-0.575564,-0.806767,-0.575564,0.231204,NaN
231,0.850611,0.149389,0.368467,0.631533,627,2267.053308,0.313422,0.055045,0.537189,0.094344,...,0.398237,0.033702,0.126876,0.345342,0.074721,-0.016221,0.126876,0.074721,0.052155,0.364534
232,0.348087,0.651913,0.654324,0.345676,696,2519.880837,0.227762,0.426562,0.120325,0.225351,...,-0.176345,0.317527,-0.579143,-0.568754,0.015854,0.313677,-0.568754,0.313677,0.255077,-0.141181


In [ ]:
## Want to test for differences between long/short among only user approved ripples:

0.025


### 2024-02-29 - 4pm - Filter the events for those meeting wcorr criteria:


In [22]:
df: pd.DataFrame = deepcopy(user_approved_ripple_df)

## INPUTS: df

min_wcorr_threshold: float = 0.33
min_wcorr_diff_threshold: float = 0.2

is_included_large_wcorr_diff = np.any((df[['wcorr_abs_diff']].abs() > min_wcorr_diff_threshold), axis=1)
is_included_high_wcorr = np.any((df[['long_best_wcorr', 'short_best_wcorr']].abs() > min_wcorr_threshold), axis=1)

df = df[is_included_high_wcorr]
df

# delta_aligned_start_t

,P_LR,P_RL,P_Long,P_Short,ripple_idx,ripple_start_t,P_Long_LR,P_Long_RL,P_Short_LR,P_Short_RL,...,long_best_pf_peak_x_pearsonr,short_best_pf_peak_x_pearsonr,wcorr_long_LR,wcorr_long_RL,wcorr_short_LR,wcorr_short_RL,long_best_wcorr,short_best_wcorr,wcorr_abs_diff,pearsonr_abs_diff
0,0.159710,0.840290,0.838429,0.161571,3,66.661598,0.133906,0.704523,0.025805,0.135767,...,-0.623696,-0.512350,0.643677,-0.747099,0.221974,-0.772675,-0.747099,-0.772675,-0.025577,0.111346
1,0.634361,0.365639,0.783215,0.216785,42,292.623671,0.496841,0.286374,0.137520,0.079265,...,-0.478802,-0.471114,0.935025,0.932670,0.863378,0.919826,0.932670,0.919826,0.012845,0.007689
2,0.493034,0.506966,0.570170,0.429830,43,304.440267,0.281113,0.289057,0.211921,0.217909,...,-0.775220,-0.640006,0.139927,0.386869,0.110647,0.088710,0.386869,0.088710,0.298158,0.135214
3,0.188260,0.811740,0.790949,0.209051,51,380.745850,0.148904,0.642045,0.039356,0.169695,...,0.855873,0.689429,-0.849326,-0.882094,-0.682860,-0.853833,-0.882094,-0.853833,0.028261,0.166444
4,0.286592,0.713408,0.711961,0.288039,79,487.205157,0.204043,0.507919,0.082550,0.205489,...,-0.938163,-0.901337,-0.941448,-0.934071,-0.799006,-0.933250,-0.934071,-0.933250,0.000822,0.036825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,0.346157,0.653843,0.671652,0.328348,623,2260.494653,0.232497,0.439155,0.113660,0.214688,...,-0.775736,-0.609679,0.285549,0.317732,-0.087196,0.581530,0.317732,0.581530,-0.263798,0.166057
229,0.113030,0.886970,0.531551,0.468449,624,2261.095905,0.060081,0.471470,0.052949,0.415500,...,NaN,NaN,-0.672955,-0.736671,-0.301308,0.025316,-0.736671,0.025316,0.711355,NaN
230,0.116618,0.883382,0.615333,0.384667,626,2265.076969,0.071759,0.543574,0.044859,0.339808,...,NaN,NaN,-0.926538,-0.806767,-0.804224,-0.575564,-0.806767,-0.575564,0.231204,NaN
232,0.348087,0.651913,0.654324,0.345676,696,2519.880837,0.227762,0.426562,0.120325,0.225351,...,-0.176345,0.317527,-0.579143,-0.568754,0.015854,0.313677,-0.568754,0.313677,0.255077,-0.141181


In [23]:
import plotly.io as pio
template: str = 'plotly_dark' # set plotl template
pio.templates.default = template
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import _helper_build_figure
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import plotly_pre_post_delta_scatter


histogram_bins = 25
concatenated_ripple_df = deepcopy(df)


# ripple_title_prefix = 'test'
# # Create a bubble chart for ripples
# ripple_num_unique_sessions: int = concatenated_ripple_df.session_name.nunique(dropna=True) # number of unique sessions, ignoring the NA entries
# # ripple_num_unique_time_bins: int = concatenated_ripple_df.time_bin_size.nunique(dropna=True)
# ripple_title_string_suffix: str = f'{ripple_num_unique_sessions} Sessions'
# ripple_title: str = f"{ripple_title_prefix} - {ripple_title_string_suffix}"
# fig_ripples = _helper_build_figure(data_results_df=concatenated_ripple_df, histogram_bins=histogram_bins, earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end, enabled_time_bin_sizes=None,
#                                     main_plot_mode='separate_row_per_session', title=ripple_title)


scatter_title = 'Sevveral Sessions'
variable_name = 'wcorr_abs_diff'
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': scatter_title, 'range_y': [0.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size'}} # , 'color': 'time_bin_size'
new_fig_ripples = plotly_pre_post_delta_scatter(data_results_df=deepcopy(concatenated_ripple_df), out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end))
new_fig_ripples



num_unique_sessions: 3, num_unique_time_bins: 1
len(pre_delta_fig.data): 1


In [25]:
# Laps test
concatenated_ripple_df = deepcopy(all_sessions_new_laps_df)

scatter_title = 'Sevveral Sessions'
variable_name = 'wcorr_abs_diff'
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': scatter_title, 'range_y': [0.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size'}} # , 'color': 'time_bin_size'
new_fig_ripples = plotly_pre_post_delta_scatter(data_results_df=deepcopy(concatenated_ripple_df), out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end))
new_fig_ripples


num_unique_sessions: 11, num_unique_time_bins: 1
len(pre_delta_fig.data): 1


# Plotting functions

In [ ]:
from pyphoplacecellanalysis.General.Model.Configs.LongShortDisplayConfig import PlottingHelpers
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import plot_across_sessions_scatter_results

# Example usage:
all_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=all_sessions_laps_df, concatenated_ripple_df=all_sessions_ripple_df,
														#    enabled_time_bin_sizes=[0.03, 0.10],
														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
														   laps_title_prefix=f"Laps", ripple_title_prefix=f"Ripples", save_figures=True, figure_save_extension=['.html','.png'])
fig_laps, fig_ripples = all_session_figures[0]


# fig_laps.show()
fig_ripples.show()
# fig_laps.write_html(f"../output/{TODAY_DAY_DATE}_AcrossSession_fig_laps.html")
# fig_ripples.write_html(f"../output/{TODAY_DAY_DATE}_AcrossSession_fig_ripples.html")

In [ ]:
## time_bin version:
all_time_bin_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=all_sessions_laps_time_bin_df, concatenated_ripple_df=all_sessions_ripple_time_bin_df,
														#    enabled_time_bin_sizes=[0.03, 0.10],
														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
														   main_plot_mode='separate_row_per_session',
														   laps_title_prefix=f"Laps_per_time_bin", ripple_title_prefix=f"Ripples_per_time_bin", save_figures=True, figure_save_extension=['.html','.png'])
fig_time_bin_laps, fig_time_bin_ripples = all_time_bin_session_figures[0]
# fig_time_bin_laps.show()
# fig_time_bin_ripples.show()

# for fig_time_bin_laps, fig_time_bin_ripples in all_time_bin_session_figures:
#     fig_time_bin_laps.show()
#     fig_time_bin_ripples.show()
#     # fig_laps.write_html("../output/2024-01-18_AcrossSession_fig_laps.html")
#     # fig_ripples.write_html("../output/2024-01-18_AcrossSession_fig_ripples.html")
#     fig_time_bin_laps.write_html("../output/2024-01-23_AcrossSession_fig_time_bin_laps.html")
#     fig_time_bin_ripples.write_html("../output/2024-01-23_AcrossSession_fig_time_bin_ripples.html")


In [ ]:
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import plot_across_sessions_scatter_results, _helper_build_figure

## Test collapsed histograms-only results:
histograms_only_all_time_bin_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=all_sessions_laps_time_bin_df, concatenated_ripple_df=all_sessions_ripple_time_bin_df,
														#    enabled_time_bin_sizes=[0.03, 0.10],
															# enabled_time_bin_sizes=[0.03, 0.058, 0.10], # [0.03 , 0.044, 0.058, 0.072, 0.086, 0.1]
														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
														   main_plot_mode='default',
														   laps_title_prefix=f"Laps_per_time_bin", ripple_title_prefix=f"Ripples_per_time_bin", save_figures=False, figure_save_extension=['.html','.png'])
histograms_only_fig_time_bin_laps, histograms_only_fig_time_bin_ripples = histograms_only_all_time_bin_session_figures[0]
histograms_only_fig_time_bin_laps.show()


In [ ]:
import plotly.subplots as sp
import plotly.graph_objs as go

from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import complete_plotly_figure

# data_results_df: pd.DataFrame = deepcopy(all_sessions_laps_df) # all_sessions_laps_time_bin_df
# histogram_bins = 'auto'
histogram_bins: int = 25

new_laps_fig = complete_plotly_figure(data_results_df=deepcopy(all_sessions_laps_df), out_scatter_fig=fig_laps, histogram_bins=histogram_bins)
new_laps_fig

# Dash-based Interactivity

In [ ]:
from typing import List, Dict, Optional, Tuple, Union
from dash import Dash, html, dash_table, dcc, callback, Output, Input, State
from dash.dash_table import DataTable, FormatTemplate
from dash.dash_table.Format import Format, Padding

import dash_bootstrap_components as dbc
import pandas as pd
from pathlib import Path
# import plotly.express as px
import plotly.io as pio
template: str = 'plotly_dark' # set plotl template
pio.templates.default = template
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import plot_across_sessions_scatter_results, _helper_build_figure
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import plot_blue_yellow_points
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import _build_dash_app

# Incorporate data
# final_csv_export_paths = {'AcrossSession_Laps_per-Epoch': Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-01-30_AcrossSession_Laps_per-Epoch.csv'),
#  'AcrossSession_Ripple_per-Epoch': Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-01-30_AcrossSession_Ripple_per-Epoch.csv'),
#  'AcrossSession_Laps_per-TimeBin': Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-01-30_AcrossSession_Laps_per-TimeBin.csv'),
#  'AcrossSession_Ripple_per-TimeBin': Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-01-30_AcrossSession_Ripple_per-TimeBin.csv')}

final_csv_export_paths = {'AcrossSession_Laps_per-Epoch': Path('/home/halechr/repos/Spike3D/output/2024-03-01_AcrossSession_Laps_per-Epoch.csv'),
 'AcrossSession_Ripple_per-Epoch': Path('/home/halechr/repos/Spike3D/output/2024-03-01_AcrossSession_Ripple_per-Epoch.csv'),
 'AcrossSession_Laps_per-TimeBin': Path('/home/halechr/repos/Spike3D/output/2024-03-01_AcrossSession_Laps_per-TimeBin.csv'),
 'AcrossSession_Ripple_per-TimeBin': Path('/home/halechr/repos/Spike3D/output/2024-03-01_AcrossSession_Ripple_per-TimeBin.csv'),
 'AcrossSession_NEW_Laps_per-Epoch': Path('/home/halechr/repos/Spike3D/output/2024-03-01_AcrossSession_NEW_Laps_per-Epoch.csv'),
 'AcrossSession_NEW_Ripple_per-Epoch': Path('/home/halechr/repos/Spike3D/output/2024-03-01_AcrossSession_NEW_Ripple_per-Epoch.csv')}

final_dfs_dict = {a_name:pd.read_csv(a_path.resolve()) for a_name, a_path in final_csv_export_paths.items()}
app = _build_dash_app(final_dfs_dict, earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end)

# Run the app
if __name__ == '__main__':
    print("Starting the app.")  # Add this line to check if it's executed
    app.run(port="8054", debug=True, use_reloader=False)  # Turn off reloader if inside Jupyter)

In [ ]:
a_df = deepcopy(final_dfs_dict['AcrossSession_Laps_per-TimeBin'])
a_df
# print(list(a_df.columns))
# ['Unnamed: 0.1', 'Unnamed: 0', 'P_LR', 'P_RL', 'P_Long', 'P_Short', 'epoch_idx', 't_bin_center', 'session_name', 'delta_aligned_start_t', 'time_bin_size']

yellow_blue_relevent_columns = ['epoch_idx', 'session_name', 'time_bin_size', 'delta_aligned_start_t', 'P_Long', 'P_Short', 'P_LR', 'P_RL']
a_df = a_df[yellow_blue_relevent_columns]

# filter by time_bin_size first:
a_single_selected_time_bin_size: float = 0.1 # time_bin_size
a_df = a_df[a_df.time_bin_size.astype(float) == a_single_selected_time_bin_size]

## Filter by selected epoch index:
selected_epoch_idxs = [5, 6, 9]
a_df = a_df[a_df.epoch_idx.isin(selected_epoch_idxs)]

# filter by selected epoch index:
a_single_epoch_row_idx: int = 0
a_single_epoch_idx = selected_epoch_idxs[a_single_epoch_row_idx]
a_single_epoch_df = a_df[a_df.epoch_idx == a_single_epoch_idx]
a_single_epoch_df

## filter by session:
a_single_session_name: str = 'kdiba_vvp01_one_2006-4-10_12-25-50'
a_single_epoch_df = a_single_epoch_df[a_single_epoch_df.session_name == a_single_session_name]
a_single_epoch_df


In [ ]:
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import plotly_plot_1D_most_likely_position_comparsions
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import plot_blue_yellow_points

# def plot_blue_yellow(a_df, selected_epoch_idxs, a_single_session_name: str):

specific_point_list = [{'session_name': 'kdiba_vvp01_one_2006-4-10_12-25-50', 'time_bin_size': 0.03, 'epoch_idx': 0, 'delta_aligned_start_t': -713.908702568122}]

# def plot_blue_yellow(a_df, selected_epoch_idxs, a_single_session_name: str):

## Takes: a_df, selected_epoch_idxs, a_single_session_name
a_df
a_single_session_name: str = 'kdiba_vvp01_one_2006-4-10_12-25-50' # #TODO 2024-01-30 16:01: - [ ] Hardcoded session:
selected_epoch_idxs = [5, 6, 9]
fig = plot_blue_yellow_points(a_df, selected_epoch_idxs=selected_epoch_idxs, a_single_session_name=a_single_session_name)
fig.show()


# Matplotlib-based versions:

In [ ]:
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import plot_histograms

matplotlib.use('Qt5Agg')

In [ ]:
# from Spike3D.PendingNotebookCode import plot_stacked_histograms

# You can use it like this:'long_best_dir_simple_pearsonr'
plot_histograms(all_sessions_laps_time_bin_df, 'Laps', 'All Sessions', "75 ms")
plot_histograms(all_sessions_ripple_time_bin_df, 'Ripples', 'All Sessions', "75 ms")

In [ ]:
## all_sessions_new_laps_df, all_sessions_new_ripple_df
plot_histograms(all_sessions_new_laps_df, 'New Laps', 'All Sessions', "25 ms")
plot_histograms(all_sessions_new_ripple_df, 'New Ripples', 'All Sessions', "25 ms")


In [ ]:
all_sessions_new_laps_df

In [ ]:
len(all_sessions_new_laps_df['session_name'].unique()) # 10 sessions



In [ ]:
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import plot_stacked_histograms

# You can use it like this:
num_unique_sessions: int = all_sessions_laps_time_bin_df.session_name.nunique(dropna=True) # number of unique sessions, ignoring the NA entries
num_unique_time_bins: int = all_sessions_laps_time_bin_df.time_bin_size.nunique(dropna=True)
plot_stacked_histograms(all_sessions_laps_time_bin_df, 'Laps', f'{num_unique_sessions} Sessions', f"{num_unique_time_bins} tbin sizes")

num_unique_sessions: int = all_sessions_ripple_time_bin_df.session_name.nunique(dropna=True) # number of unique sessions, ignoring the NA entries
num_unique_time_bins: int = all_sessions_ripple_time_bin_df.time_bin_size.nunique(dropna=True)
plot_stacked_histograms(all_sessions_ripple_time_bin_df, 'Ripples', f'{num_unique_sessions} Sessions', f"{num_unique_time_bins} tbin sizes")

## 2024-02-20 - New Pearsonr analysis

In [ ]:
new_column_names = ['long_LR_pf_peak_x_pearsonr', 'long_RL_pf_peak_x_pearsonr', 'short_LR_pf_peak_x_pearsonr', 'short_RL_pf_peak_x_pearsonr']


def _add_best_direction_columns(_temp_all_sessions_new_ripple_df):
    """ Get the most-likely direction to determine which correlation to use
    
    """
    ## Get the most-likely direction to determine which correlation to use
    # _temp_all_sessions_new_ripple_df['best_dir_simple_pearsonr'] = pd.NA

    # _temp_all_sessions_new_ripple_df[_temp_all_sessions_new_ripple_df['P_LR'] > 0.5] # LR-likely plots

    ## Adds two new columns using the best direction as determined by the decoder: ['long_best_dir_simple_pearsonr', 'short_best_dir_simple_pearsonr']
    # Using numpy where
    _temp_all_sessions_new_ripple_df['long_best_dir_simple_pearsonr'] = np.where((_temp_all_sessions_new_ripple_df['P_LR'] > 0.5), _temp_all_sessions_new_ripple_df['long_LR_pf_peak_x_pearsonr'], _temp_all_sessions_new_ripple_df['long_RL_pf_peak_x_pearsonr'])
    _temp_all_sessions_new_ripple_df['short_best_dir_simple_pearsonr'] = np.where((_temp_all_sessions_new_ripple_df['P_LR'] > 0.5), _temp_all_sessions_new_ripple_df['short_LR_pf_peak_x_pearsonr'], _temp_all_sessions_new_ripple_df['short_RL_pf_peak_x_pearsonr'])
    ## Define a difference of the absolutes
    _temp_all_sessions_new_ripple_df['long_short_diff_best_dir_simple_pearsonr'] = (_temp_all_sessions_new_ripple_df['long_best_dir_simple_pearsonr'].abs() - _temp_all_sessions_new_ripple_df['short_best_dir_simple_pearsonr'].abs())
    return _temp_all_sessions_new_ripple_df


## Drop the dang NaN columns:
_temp_all_sessions_new_ripple_df= _add_best_direction_columns(all_sessions_new_ripple_df.dropna(subset=new_column_names))
_temp_all_sessions_new_laps_df= _add_best_direction_columns(all_sessions_new_laps_df.dropna(subset=new_column_names))
_temp_all_sessions_new_laps_df

In [ ]:
['long_best_dir_simple_pearsonr_abs']

plot_histograms(_temp_all_sessions_new_laps_df, 'Laps', 'Some Sessions', "25 ms", column='long_best_dir_simple_pearsonr')

In [ ]:
## Get only high-confidence replays:
_temp_high_confidence_ripple_df = _temp_all_sessions_new_ripple_df[_temp_all_sessions_new_ripple_df['P_Long'] > 0.85]
# plot_histograms(_temp_high_confidence_ripple_df, 'Ripple', 'Some Sessions', "25 ms", column='long_best_dir_simple_pearsonr')
plot_histograms(_temp_high_confidence_ripple_df, 'Ripple', 'Some Sessions', "25 ms", column='long_best_dir_simple_pearsonr')



In [ ]:
# _temp_all_sessions_new_ripple_df

plt.scatter(x='delta_aligned_start_t', y='long_best_dir_simple_pearsonr', data=_temp_all_sessions_new_ripple_df)
plt.show()

In [ ]:
# plt.scatter(x='delta_aligned_start_t', y='long_best_dir_simple_pearsonr', data=_temp_all_sessions_new_ripple_df)

# pile
plt.figure()
plt.ion()
# plt.scatter(x=_temp_all_sessions_new_ripple_df['delta_aligned_start_t'], y=_temp_all_sessions_new_ripple_df['long_best_dir_simple_pearsonr'])
plt.scatter(x=_temp_all_sessions_new_ripple_df['delta_aligned_start_t'], y=_temp_all_sessions_new_ripple_df['long_short_diff_best_dir_simple_pearsonr'])
plt.show()


In [ ]:
# plot_histograms(_temp_all_sessions_new_ripple_df, 'Ripple', 'Some Sessions', "25 ms", column='long_short_diff_best_dir_simple_pearsonr')





In [ ]:
import scipy.stats as stats

data_results_df: pd.DataFrame = deepcopy(_temp_all_sessions_new_ripple_df)
column = 'long_short_diff_best_dir_simple_pearsonr' # REJECT (t-statistic: 2.0498849677801947, p-value: 0.04048207344769608)
# column = 'short_best_dir_simple_pearsonr' # FAIL
# column = 'long_best_dir_simple_pearsonr' # FAIL (t-statistic: 0.5238142457664882, p-value: 0.600455893577746)

# get the pre-delta epochs
pre_delta_df = data_results_df[data_results_df['delta_aligned_start_t'] <= 0]
post_delta_df = data_results_df[data_results_df['delta_aligned_start_t'] > 0]

# Perform the t-test for two independent samples
# T_result = stats.ttest_ind(pre_delta_df[column].to_numpy(), post_delta_df[column].to_numpy(), equal_var=False)
T_result = stats.ttest_ind(pre_delta_df[column].to_numpy(), post_delta_df[column].to_numpy(), equal_var=False) # Welch's t-test
t_statistic, p_value = T_result

print(f"t-statistic: {t_statistic}")
print(f"p-value: {p_value}")

# Decision making
alpha = 0.05  # commonly used significance level
if p_value < alpha:
    print("Reject the null hypothesis. There is a significant difference between the two group means.")
else:
    print("Fail to reject the null hypothesis. There is no significant difference between the two group means.")


In [ ]:

all_sessions_new_ripple_df


In [ ]:
plt.show()

In [ ]:
time_bin_sizes: int = all_sessions_laps_time_bin_df['time_bin_size'].unique()
time_bin_sizes

In [ ]:
all_sessions_laps_time_bin_df.hist(column='P_Long')
plt.show()

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import plot_all_epoch_bins_marginal_predictions
from attrs import define, field, Factory
from neuropy.utils.result_context import IdentifyingContext
matplotlib.use('Qt5Agg')
# %matplotlib inline
# %matplotlib auto

# Switch to the desired interactivity mode
plt.interactive(True)


@define(slots=False)
class TwoCSV:
	""" simple class wrapper to emulate the object that holds the other dfs """
	laps_all_epoch_bins_marginals_df = field()
	ripple_all_epoch_bins_marginals_df = field()

save_figure = True
def _perform_write_to_file_callback(final_context, fig):
	print(f'final_context: {final_context}')
	# if save_figure:
	# 	fig.save_fig(
	# 	# return owning_pipeline_reference.output_figure(final_context, fig)
	# else:
	# 	pass # do nothing, don't save
	
# all_sessions_laps_time_bin_df

collector = plot_all_epoch_bins_marginal_predictions(TwoCSV(laps_all_epoch_bins_marginals_df=all_sessions_laps_df, ripple_all_epoch_bins_marginals_df=all_sessions_ripple_df), t_start=None, t_split=0.0, t_end=None,
													active_context=IdentifyingContext(), perform_write_to_file_callback=_perform_write_to_file_callback)
collector


result = tuple(collector.figures)
result[0]

In [ ]:
result[0].show()

In [ ]:
result[1]

In [ ]:
all_sessions_laps_df, all_sessions_ripple_df

# BAD 2024-02-08 - Temporal Timecourse of Long/Short Replays after Delta using cross-correlations and auto-correlations

In [ ]:
from pyphocorehelpers.indexing_helpers import partition_df

def _split_is_post_delta(df):
    df['is_post_delta'] = (df['delta_aligned_start_t'] > 0.0)
    return partition_df(df, partitionColumn='is_post_delta')[-1] # [-1] gets the actual dataframe list

def _split_long_likely(df):
    df['is_long_likely'] = (df['P_Long'] > 0.5)
    return partition_df(df, partitionColumn='is_long_likely')[-1] # [-1] gets the actual dataframe list

def _split_on_pre_post_delta_and_likely_track(df):
    pre_delta_ripple_df, post_delta_ripple_df = _split_is_post_delta(df)
    pre_delta_long_likely_ripple_df, pre_delta_short_likely_ripple_df = _split_long_likely(pre_delta_ripple_df)
    post_delta_long_likely_ripple_df, post_delta_short_likely_ripple_df = _split_long_likely(post_delta_ripple_df)
    return (pre_delta_ripple_df, post_delta_ripple_df), (pre_delta_long_likely_ripple_df, pre_delta_short_likely_ripple_df), (post_delta_long_likely_ripple_df, post_delta_short_likely_ripple_df)


# only allow one time bin size
# active_time_bin_size = 0.10

# active_time_bin_size = 0.025
active_time_bin_size = 0.03

## Get only those after the delta, which occurs at t=0 relative to delta_aligned_start_t:
filtered_all_sessions_laps_df = all_sessions_laps_df[all_sessions_laps_df['time_bin_size'] == active_time_bin_size].copy()
filtered_all_sessions_laps_df['is_long_likely'] = (filtered_all_sessions_laps_df['P_Long'] > 0.5)
(pre_delta_laps_df, post_delta_laps_df), (pre_delta_long_likely_laps_df, pre_delta_short_likely_laps_df), (post_delta_long_likely_laps_df, post_delta_short_likely_laps_df) = _split_on_pre_post_delta_and_likely_track(filtered_all_sessions_laps_df)

filtered_all_sessions_ripple_df = all_sessions_ripple_df[all_sessions_ripple_df['time_bin_size'] == active_time_bin_size].copy().drop(columns=['time_bin_size'])
filtered_all_sessions_ripple_df['is_long_likely'] = (filtered_all_sessions_ripple_df['P_Long'] > 0.5)
(pre_delta_ripple_df, post_delta_ripple_df), (pre_delta_long_likely_ripple_df, pre_delta_short_likely_ripple_df), (post_delta_long_likely_ripple_df, post_delta_short_likely_ripple_df) = _split_on_pre_post_delta_and_likely_track(filtered_all_sessions_ripple_df)


In [ ]:
laps_df_dict = {
	"all_post_delta_laps": filtered_all_sessions_laps_df,
	"long_likely_laps_df": long_likely_laps_df,
	"short_likely_laps_df": short_likely_laps_df,
}
	
ripples_df_dict = {
	"all_post_delta_ripple": filtered_all_sessions_ripple_df,
	"long_likely_ripple_df": long_likely_ripple_df,
	"short_likely_ripple_df": short_likely_ripple_df,
}

all_sessions_laps_df, all_sessions_ripple_df
all_sessions_laps_time_bin_df, all_sessions_ripple_time_bin_df
earliest_delta_aligned_t_start, latest_delta_aligned_t_end

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
import matplotlib.pyplot as plt
# import proplot as pplt
# %matplotlib inline
# %matplotlib auto
%matplotlib qt


In [ ]:
def plot_laps(laps_df_dict):
    """
    Consolidates the laps plots into a single figure
    Args:
        laps_df_dict (dict): A dictionary with keys as plot titles and values as DataFrame with 'delta_aligned_start_t' column
    """
    fig, axs = plt.subplots(3, figsize=(10, 15))
    fig.suptitle('Laps Plots')
    
    for i, (title, df) in enumerate(laps_df_dict.items()):
        pd.plotting.autocorrelation_plot(df.delta_aligned_start_t, ax=axs[i])
        axs[i].set_title(title)
        axs[i].set_xlabel('Lag (seconds)')
        
    plt.tight_layout()
    plt.subplots_adjust(top=0.95)
    plt.show()

def plot_ripples(ripples_df_dict):
    """
    Consolidates the ripple plots into a single figure
    Args:
        ripples_df_dict (dict): A dictionary with keys as plot titles and values as DataFrame with 'delta_aligned_start_t' column
    """
    fig, axs = plt.subplots(3, figsize=(10, 15))
    fig.suptitle('Ripples Plots')
    
    for i, (title, df) in enumerate(ripples_df_dict.items()):
        pd.plotting.autocorrelation_plot(df.delta_aligned_start_t, ax=axs[i])
        axs[i].set_title(title)
        axs[i].set_xlabel('Lag (seconds)')
        
    plt.tight_layout()
    plt.subplots_adjust(top=0.95)
    plt.show()

## BODY
def _plot_all_signal_correlations(filtered_all_sessions_laps_df, long_likely_laps_df, short_likely_laps_df, filtered_all_sessions_ripple_df, long_likely_ripple_df, short_likely_ripple_df):
    laps_df_dict = {
        "all_post_delta_laps": filtered_all_sessions_laps_df,
        "long_likely_laps_df": long_likely_laps_df,
        "short_likely_laps_df": short_likely_laps_df,
    }
    plot_laps(laps_df_dict)
        
    ripples_df_dict = {
        "all_post_delta_ripple": filtered_all_sessions_ripple_df,
        "long_likely_ripple_df": long_likely_ripple_df,
        "short_likely_ripple_df": short_likely_ripple_df,
    }
    plot_ripples(ripples_df_dict)



## Inputs: pre_delta_long_likely_laps_df, pre_delta_short_likely_laps_df, pre_delta_long_likely_ripple_df, pre_delta_short_likely_ripple_df
# long_likely_laps_df = pre_delta_long_likely_laps_df.copy()
# short_likely_laps_df = pre_delta_short_likely_laps_df.copy()

# long_likely_ripple_df = pre_delta_long_likely_ripple_df.copy()
# short_likely_ripple_df = pre_delta_short_likely_ripple_df.copy()

## Inputs: pre_delta_long_likely_laps_df, pre_delta_short_likely_laps_df, pre_delta_long_likely_ripple_df, pre_delta_short_likely_ripple_df

# _plot_all_signal_correlations(filtered_all_sessions_laps_df, long_likely_laps_df, short_likely_laps_df, filtered_all_sessions_ripple_df, long_likely_ripple_df, short_likely_ripple_df)
# plt.suptitle('Pre-$\Delta$ Time Correlations')

# _plot_all_signal_correlations(filtered_all_sessions_laps_df, pre_delta_long_likely_laps_df.copy(), pre_delta_short_likely_laps_df.copy(), filtered_all_sessions_ripple_df, pre_delta_long_likely_ripple_df.copy(), pre_delta_short_likely_ripple_df.copy())
_plot_all_signal_correlations(filtered_all_sessions_laps_df, post_delta_long_likely_laps_df.copy(), post_delta_short_likely_laps_df.copy(), filtered_all_sessions_ripple_df, post_delta_long_likely_ripple_df.copy(), post_delta_short_likely_ripple_df.copy())
plt.suptitle('Post-$\Delta$ Time Correlations - All Sessions')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# sns.kdeplot(filtered_all_sessions_ripple_df['delta_aligned_start_t'])
# sns.kdeplot(filtered_all_sessions_ripple_df['delta_aligned_start_t'])
# sns.kdeplot(filtered_all_sessions_ripple_df, x='delta_aligned_start_t', hue='is_post_delta', y='is_long_likely')
sns.kdeplot(filtered_all_sessions_ripple_df, x='delta_aligned_start_t', hue='is_long_likely')
plt.show()

In [ ]:
sns.kdeplot(long_likely_ripple_df, x='delta_aligned_start_t')
plt.show()

In [ ]:
sns.kdeplot(short_likely_ripple_df['delta_aligned_start_t'])
plt.show()

In [ ]:
sns.kdeplot(post_delta_long_likely_ripple_df['delta_aligned_start_t'])
plt.show()

In [ ]:
sns.kdeplot(post_delta_short_likely_ripple_df['delta_aligned_start_t'])

In [ ]:
# Assuming series1 and series2 are your two pandas Series
correlation = long_likely_ripple_df.delta_aligned_start_t.corr(short_likely_ripple_df.delta_aligned_start_t)
print('Correlation: ', correlation)

# # Scatter plot
# plt.scatter(long_likely_ripple_df.delta_aligned_start_t, short_likely_ripple_df.delta_aligned_start_t)
# plt.title('Scatter Plot')
# plt.xlabel('Series 1')
# plt.ylabel('Series 2')
# plt.show()

In [ ]:

 # delta_aligned_start_t

# filtered_all_sessions_laps_df.delta_aligned_start_t.autocorr()

# plot_acf(filtered_all_sessions_laps_df.delta_aligned_start_t)
plt.show()

In [ ]:
plt.close('all')

In [ ]:
import pycorrelate as pyc

# compute lags in timestamp units (not in seconds!)
# to avoid floating point inacuracies
# bins_per_dec = 10
# unit = np.timedelta64()

# bins = pyc.make_loglags(1, 8, bins_per_dec)[bins_per_dec//2:]
# Convert the float array to timedelta64 by multiplying with '1s'
# bins = (bins * 1e9).astype('timedelta64[ns]')

unit = 1e9
# convert_to_time_units = lambda t: (t * 1e9).astype('timedelta64[ns]')
convert_to_time_units = lambda t: (t) #.astype('timedelta64[ns]')

# compute lags in sec. then convert to timestamp units
# bins = pyc.make_loglags(-1, 5, 20)
# bins = pyc.make_loglags(-1, 5, 5)
bins = pyc.make_loglags(-1, 2, 5)

# Convert the float array to timedelta64 by multiplying with '1s'
# bins = (bins * 1e9).astype('timedelta64[ns]')
bins = convert_to_time_units(bins)

long_likely_epoch_times = convert_to_time_units(deepcopy(long_likely_laps_df['delta_aligned_start_t'].to_numpy()))
short_likely_epoch_times = convert_to_time_units(deepcopy(short_likely_laps_df['delta_aligned_start_t'].to_numpy()))

# long_likely_epoch_times = convert_to_time_units(deepcopy(long_likely_ripples_df['delta_aligned_start_t'].to_numpy()))
# short_likely_epoch_times = convert_to_time_units(deepcopy(short_likely_ripples_df['delta_aligned_start_t'].to_numpy()))

long_likely_autocorrelation = pyc.pcorrelate(long_likely_epoch_times, long_likely_epoch_times, bins)
short_likely_autocorrelation = pyc.pcorrelate(short_likely_epoch_times, short_likely_epoch_times, bins)
long_short_likely_cross_correlation = pyc.pcorrelate(long_likely_epoch_times, short_likely_epoch_times, bins)

long_likely_autocorrelation
short_likely_autocorrelation
long_short_likely_cross_correlation

In [ ]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook"

# Convert data to DataFrame
t_bins = deepcopy(bins.astype('float'))  # convert timedelta64[ns] to float
t_bins = (t_bins[1:] + t_bins[:-1]) / 2

df1 = pd.DataFrame({'Bins': t_bins, 'Correlation': long_likely_autocorrelation, 'Type': 'Long Autocorrelation'})
df2 = pd.DataFrame({'Bins': t_bins, 'Correlation': short_likely_autocorrelation, 'Type': 'Short Autocorrelation'})
df3 = pd.DataFrame({'Bins': t_bins, 'Correlation': long_short_likely_cross_correlation, 'Type': 'Cross Correlation'})

df = pd.concat([df1, df2, df3])

# Plot using Plotly Express
fig = px.line(df, x="Bins", y="Correlation", color="Type", facet_row="Type", 
              labels={'Bins': 'Bins (seconds)', 'Correlation': 'Correlation'},
              title='Correlation vs Bins')
# fig.update_xaxes(type="log")
fig.update_yaxes(type="log")

fig

In [ ]:
G = long_short_likely_cross_correlation.copy()
# bins_unit = bins*unit
bins_unit = bins.copy()
fig, ax = plt.subplots(figsize=(10, 6))
plt.plot(bins_unit, np.hstack((G[:1], G)), drawstyle='steps-pre')
plt.xlabel('Time (s)')
#for x in bins[1:]: plt.axvline(x*unit, lw=0.2)  # to mark bins
plt.grid(True); plt.grid(True, which='minor', lw=0.3)
plt.xscale('log')
# plt.xlim(30e-9, 2)
plt.title('Cross-correlation of Long and Short')

long_short_likely_cross_correlation

In [ ]:
# Tweak here matplotlib style
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['font.sans-serif'].insert(0, 'Arial')
mpl.rcParams['font.size'] = 10
# %config InlineBackend.figure_format = 'retina'
# %matplotlib qt
# matplotlib.use('Qt5Agg')

def plot_correlate(G):
    fig, ax = plt.subplots(figsize=(10, 6))
    plt.plot(bins, np.hstack((G[:1], G)), drawstyle='steps-pre')
    plt.xlabel('Time (s)')
    #for x in bins[1:]: plt.axvline(x*unit, lw=0.2)  # to mark bins
    plt.grid(True); plt.grid(True, which='minor', lw=0.3)
    plt.xscale('log')
    plt.xlim(30e-9, 2)
    return fig, ax

# fig, ax = plot_correlate(long_likely_autocorrelation)
fig, ax = plot_correlate(long_short_likely_cross_correlation)
# fig.show()
fig